In [4]:
m3.factor_profit_analyze()

TypeError: Invalid comparison between dtype=datetime64[ns] and date

In [2]:
# 本代码由可视化策略环境自动生成 2021年8月16日16:29
# 本代码单元只能在可视化模式下编辑。您也可以拷贝代码，粘贴到新建的代码单元或者策略，然后修改。


# 回测引擎：初始化函数，只执行一次
def m3_initialize_bigquant_run(context):

    # 系统已经设置了默认的交易手续费和滑点，要修改手续费可使用如下函数
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))

# 回测引擎：每日数据处理函数，每天执行一次
def m3_handle_data_bigquant_run(context, data):
    # 获取今日的日期字符串
    today = data.current_dt.strftime('%Y-%m-%d')  
    # 通过positions对象，使用列表生成式的方法获取目前持仓的股票列表
    stock_hold_now = {e.symbol: p.amount * p.last_sale_price
                 for e, p in context.portfolio.positions.items()}
    
    try:
        buy_stock = context.daily_stock_buy[today]  # 当日符合买入条件的股票
    except:
        buy_stock=[]  # 如果没有符合条件的股票，就设置为空
    
    try:
        sell_stock = context.daily_stock_sell[today]  # 当日符合卖出条件的股票
    except:
        sell_stock=[] # 如果没有符合条件的股票，就设置为空
    
    # 需要卖出的股票:已有持仓中符合卖出条件的股票
    stock_to_sell = [ i for i in stock_hold_now if i in sell_stock ]
    
    # 如果有卖出信号
    if len(stock_to_sell)>0:
        for instrument in stock_to_sell:
            # 将标的转化为equity格式以便下单操作
            sid = context.symbol(instrument) 
            # 获取股票持仓
            cur_position = context.portfolio.positions[sid].amount
            # 如果有股票持仓并且处于可交易状态，则下单卖出
            if cur_position > 0 and data.can_trade(sid):
                # 通过order_target_percent下单至指定仓位比例(通常0到1之间)
                context.order_target_percent(sid, 0) # 卖出操作
   
    # 如果有买入信号/需要调整持仓
    if len(buy_stock )>0:
        # 每只股票的比重为等资金比例持有
        weight = 1/len(buy_stock )
        for instrument in buy_stock :
            # 将标的转化为equity格式以便下单操作
            sid = context.symbol(instrument)
            if  data.can_trade(sid):
                context.order_target_percent(sid, weight) # 调仓至指定的比例
# 回测引擎：准备数据，只执行一次
def m3_prepare_bigquant_run(context):
    # 加载预测数据
    df = context.options['data'].read_df()

    # 函数：求满足开仓条件的股票列表
    def open_pos_con(df):
        return list(df[df['buy_condition']>0].instrument)

    # 函数：求满足平仓条件的股票列表
    def close_pos_con(df):
        return list(df[df['sell_condition']>0].instrument)

    # 每日买入股票的数据框
    context.daily_stock_buy= df.groupby('date').apply(open_pos_con)
    # 每日卖出股票的数据框
    context.daily_stock_sell= df.groupby('date').apply(close_pos_con)

m1 = M.input_features.v1(
    features="""# #号开始的表示注释
# 多个特征，每行一个，可以包含基础特征和衍生特征
buy_condition=where(mean(close_0,5)>mean(close_0,50),1,0)
sell_condition=where(mean(close_0,5)<mean(close_0,50),1,0)""",
    m_cached=False
)

m2 = M.instruments.v2(
    start_date=T.live_run_param('trading_date', '2012-05-29'),
    end_date=T.live_run_param('trading_date', '2020-10-28'),
    market='CN_STOCK_A',
    instrument_list='603363.SHA',
    max_count=0
)

m7 = M.general_feature_extractor.v7(
    instruments=m2.data,
    features=m1.data,
    start_date='',
    end_date='',
    before_start_days=30
)

m8 = M.derived_feature_extractor.v3(
    input_data=m7.data,
    features=m1.data,
    date_col='date',
    instrument_col='instrument',
    drop_na=False,
    remove_extra_columns=False
)

m4 = M.dropnan.v2(
    input_data=m8.data
)

m3 = M.trade.v4(
    instruments=m2.data,
    options_data=m4.data,
    start_date='',
    end_date='',
    initialize=m3_initialize_bigquant_run,
    handle_data=m3_handle_data_bigquant_run,
    prepare=m3_prepare_bigquant_run,
    volume_limit=0.025,
    order_price_field_buy='open',
    order_price_field_sell='open',
    capital_base=1000000,
    auto_cancel_non_tradable_orders=True,
    data_frequency='daily',
    price_type='后复权',
    product_type='股票',
    plot_charts=True,
    backtest_only=False,
    benchmark=''
)


[2021-08-16 16:14:43.112624] INFO: moduleinvoker: input_features.v1 开始运行..

[2021-08-16 16:14:43.144201] INFO: moduleinvoker: input_features.v1 运行完成[0.031597s].

[2021-08-16 16:14:43.149990] INFO: moduleinvoker: instruments.v2 开始运行..

[2021-08-16 16:14:43.176685] INFO: moduleinvoker: instruments.v2 运行完成[0.026686s].

[2021-08-16 16:14:43.294464] INFO: moduleinvoker: general_feature_extractor.v7 开始运行..

...

[2021-08-16 16:15:24.643069] INFO: Performance: Simulated 2048 trading days out of 2048.

[2021-08-16 16:15:24.644551] INFO: Performance: first open: 2012-05-29 09:30:00+00:00

[2021-08-16 16:15:24.645634] INFO: Performance: last close: 2020-10-28 15:00:00+00:00

[2021-08-16 16:15:29.450315] INFO: moduleinvoker: backtest.v8 运行完成[39.867185s].

[2021-08-16 16:15:29.451875] INFO: moduleinvoker: trade.v4 运行完成[41.490804s].